# Module 3 - Update Feature Group (Optional notebook)

**Note:** Please set kernel to `Python 3 (Data Science)` and select instance to `ml.t3.medium`

---

## Contents

1. [Setup](#setup)
1. [Explore existing customer feature group and data](#explore-customer-fg)
1. [Update customer feature group](#update-customer-fg)
1. [Ingest data into customer feature group](#ingest-customer-fg)
1. [Prepare training data set to retrain model](#model-training-data)
1. [Retrain XG Boost model](#retrain-xg-boost)
1. [Test model performance against test data](#real-time-inference)

---

In this notebook, we will illustrate how to modify a feature group using boto3 API and then ingest data into modified feature group. We will cover the following aspects:

* Look at existing data from customer feature group
* Modify customer feature group to add "has_kids" feature and ingest sample data
* Verify for a customer record that data has been ingested
* Athena query for dataset extraction to prepare data set for retraining(programmatically using SageMaker SDK)
* Retrain an XGBoost model similar to what we did in the notebook `m3_nb1_model_training.ipynb` in this module
* Test by deploying the model and predicting against test data
* Cleanup resources



## Setup
<a id='setup'></a>

In [1]:
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.serializers import CSVSerializer
from sagemaker.inputs import TrainingInput
from sagemaker.predictor import Predictor
from datetime import datetime, timezone, date
from random import randint
import pandas as pd
import numpy as np
import subprocess
import sagemaker
import importlib
import logging
import time
import sys
import boto3
import os

In [ ]:
if sagemaker.__version__ < '2.48.1':
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker==2.48.1'])
    importlib.reload(sagemaker)

In [ ]:
if boto3.__version__ < '1.24.23':
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'boto3==1.24.23'])
    importlib.reload(boto3)

In [3]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [ ]:
logger.info(f'Using SageMaker version: {sagemaker.__version__}')
logger.info(f'Using Pandas version: {pd.__version__}')
logger.info(f'Using boto3 version: {boto3.__version__}')

In [ ]:
import pprint
pretty_printer = pprint.PrettyPrinter(indent=4)

In [ ]:
!mkdir ../data/retrain

## Essentials

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
logger.info(f'Default S3 bucket = {default_bucket}')
prefix = 'sagemaker-feature-store'

In [ ]:
region = sagemaker_session.boto_region_name

In [ ]:
boto_session = boto3.Session(region_name=region)
sagemaker_runtime = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)
s3 = boto_session.resource('s3')


In [ ]:
def generate_event_timestamp():
    # naive datetime representing local time
    naive_dt = datetime.now()
    # take timezone into account
    aware_dt = naive_dt.astimezone()
    # time in UTC
    utc_dt = aware_dt.astimezone(timezone.utc)
    # transform to ISO-8601 format
    event_time = utc_dt.isoformat(timespec='milliseconds')
    event_time = event_time.replace('+00:00', 'Z')
    return event_time

### Explore existing feature definition and the data set
<a id='explore-customer-fg'></a>

Retrieve variables stored in previous notebooks for feature group names

In [ ]:
# Retreive FG names
%store -r customers_feature_group_name
%store -r products_feature_group_name
%store -r orders_feature_group_name
logger.info(f'Customers FG: {customers_feature_group_name}')
logger.info(f'Products FG: {products_feature_group_name}')
logger.info(f'Orders FG: {orders_feature_group_name}')

In [ ]:
customers_fg = FeatureGroup(name=customers_feature_group_name, sagemaker_session=sagemaker_session)  
products_fg = FeatureGroup(name=products_feature_group_name, sagemaker_session=sagemaker_session)
orders_fg = FeatureGroup(name=orders_feature_group_name, sagemaker_session=sagemaker_session)

Verify record exists in Customer Feature Group for a random customer_id

In [ ]:
customer_id =  f'C{randint(1, 10000)}'
logger.info(f'customer_id={customer_id}') 

In [ ]:
feature_record = featurestore_runtime.get_record(FeatureGroupName=customers_feature_group_name, 
                                                        RecordIdentifierValueAsString=customer_id)
feature_record

In [ ]:
describe_feature_group_result = sagemaker_runtime.describe_feature_group(
    FeatureGroupName=customers_feature_group_name
)
pretty_printer.pprint(describe_feature_group_result)

### Update feature group and ingest data
<a id='update-customer-fg' />

The sample product set that we have are spread out across different categories - baby products, candies, cleaning products etc. So let us assume that a customer *“having kids or not”* is defintely an indicator of them buying baby and kids products. Lets go ahead and modify the customer feature group to add this new feature.

In [ ]:
# Call UpdateFeatureGroup with feature addition(s)
sagemaker_runtime.update_feature_group(
    FeatureGroupName=customers_feature_group_name,
    FeatureAdditions=[
        {"FeatureName": "has_kids", "FeatureType": "Integral"}
    ]
)

In [ ]:
time.sleep(10)

In [ ]:
describe_feature_group_result = sagemaker_runtime.describe_feature_group(
    FeatureGroupName=customers_feature_group_name
)
pretty_printer.pprint(describe_feature_group_result)

### Prepare "has_kids" feature data and ingest data again into customer feature group. 
<a id='ingest-customer-fg' />

We retrieve the customer data from the csv and randomy generate 0 or 1 for "has_kids" feature and ingest into feature group

In [ ]:
customers_df = pd.read_csv('.././data/transformed/customers.csv')
customers_df.head(5)

In [ ]:
customers_df['has_kids']=np.random.randint(0, 2, customers_df.shape[0])

In [ ]:
customers_df=customers_df.drop(['event_time'],axis=1)

In [ ]:
event_timestamps = [generate_event_timestamp() for _ in range(len(customers_df))]
customers_df['event_time'] = event_timestamps
customers_df.head(5)

In [ ]:
%%time
customers_fg.ingest(data_frame=customers_df, max_processes=16, wait=True)

Verify this new data for the randomly generated customer_id

In [ ]:
get_record_result = featurestore_runtime.get_record(
    FeatureGroupName=customers_feature_group_name,
    RecordIdentifierValueAsString=customer_id
)
pretty_printer.pprint(get_record_result)

Let us run Athena query to verify offline store

In [ ]:
customers_query = customers_fg.athena_query()
customers_table = customers_query.table_name

In [ ]:
output_location = f's3://{default_bucket}/{prefix}/query_results/'

In [ ]:
customers_query.run(query_string=query_string,output_location=output_location)
customers_query.wait()
athena_df = customers_query.as_dataframe()
athena_df.head()

As we see from the above step, it is very easy now to modify an existing feature group, add new features, and ingest data.

In [ ]:
customers_df.to_csv('.././data/transformed/customers_has_kids.csv', index=False)

## Optional steps
From here on in this notebook, we use the data that has the new feature "has_kids" and train the model again with the data, deploy the model and test it against sample data. The intention is not to prove that model performance improves (mind you this is sample data!) but to show a real life use case where modified feature groups can be used for training.

### Verify offline store

#### Query in Athena console

If it is for the first time we are launching Athena in AWS console we need to click on `Get Started` button and then before we run the first query we need to set up a query results location in Amazon S3. 

After setting the query results location, on the left panel we need to select the `AwsDataCatalog` as Data source and the `sagemaker_featurestore` as Database.

We can run now run a query for the offline feature store data in Athena. To select the entries from the orders feature group we use the following SQL query. You will need to replace the orders table name with the corresponded value from your environment.

```sql
select * from "<customers-table>"
limit 100
```

![Customers offline data](../images/m3_nb4_athena_query.png "Customers Offline Data")

### Prepare model training dataset
<a id='model-training-data' />

Prepare train, test and validation data 

In [ ]:
products_query = products_fg.athena_query()
products_table = products_query.table_name

orders_query = orders_fg.athena_query()
orders_table = orders_query.table_name

To prepare training, validation and test data, we run an Athena query against offline feature store and get records for which "has_kids" has been populated. Why do we do this? Because offline feature store has historical records, we want only the latest ingested data that has "has_kids" populated for retraining our model.

In [ ]:
query_string = f'SELECT * FROM "{customers_table}", "{products_table}", "{orders_table}" ' \
               f'WHERE ("{orders_table}"."customer_id" = "{customers_table}"."customer_id") ' \
               f'AND ("{orders_table}"."product_id" = "{products_table}"."product_id")' \
               f'AND ("{customers_table}"."has_kids" is not null)'
query_string

In [ ]:
orders_query.run(query_string=query_string, output_location=output_location)
orders_query.wait()
joined_df = orders_query.as_dataframe()
joined_df.head()

In [ ]:
joined_df.shape

In [ ]:
model_df = joined_df.drop(['order_id', 
                           'customer_id', 
                           'product_id', 
                           'event_time', 
                           'write_time', 
                           'api_invocation_time', 
                           'is_deleted', 
                           'product_id.1', 
                           'event_time.1', 
                           'write_time.1', 
                           'api_invocation_time.1', 
                           'is_deleted.1', 
                           'customer_id.1', 
                           'purchase_amount',
                           'event_time.2', 
                           'n_days_since_last_purchase',
                           'write_time.2', 
                           'api_invocation_time.2', 
                           'is_deleted.2'], axis=1)

In [ ]:
model_df.head(5)

In [ ]:
first_column = model_df.pop('is_reordered')
model_df.insert(0, 'is_reordered', first_column)
model_df.head()

In [ ]:
model_df.to_csv('.././data/train/transformed_has_kids.csv', index=False)

### Retrain the XGBoost model with the update feature group
<a id='retrain-xg-boost' />

Now lets train the model again with this new data set

In [ ]:
train_df, validation_df, test_df = np.split(model_df.sample(frac=1, random_state=123), [int(.7*len(model_df)), int(.9*len(model_df))])

In [ ]:
train_df.shape

In [ ]:
validation_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.to_csv('../data/retrain/train.csv', index=False)
validation_df.to_csv('../data/retrain/validation.csv', index=False)
test_df.to_csv('../data/retrain/test.csv', index=False)

In [ ]:
s3.Bucket(default_bucket).Object(os.path.join(prefix, 'retrain/train.csv')).upload_file('.././data/retrain/train.csv')
s3.Bucket(default_bucket).Object(os.path.join(prefix, 'retrain/validation.csv')).upload_file('.././data/retrain/validation.csv')
s3.Bucket(default_bucket).Object(os.path.join(prefix, 'retrain/test.csv')).upload_file('.././data/retrain/test.csv')

In [ ]:
train_set_location = 's3://{}/{}/retrain/'.format(default_bucket, prefix)
validation_set_location = 's3://{}/{}/retrain/'.format(default_bucket, prefix)
test_set_location = 's3://{}/{}/retrain/'.format(default_bucket, prefix)

In [ ]:
train_set_pointer = TrainingInput(s3_data=train_set_location, content_type='csv')
validation_set_pointer = TrainingInput(s3_data=validation_set_location, content_type='csv')
test_set_pointer = TrainingInput(s3_data=test_set_location, content_type='csv')

In [ ]:
container_uri = sagemaker.image_uris.retrieve(region=boto_session.region_name, 
                                              framework='xgboost', 
                                              version='1.0-1', 
                                              image_scope='training')

In [ ]:
xgb = sagemaker.estimator.Estimator(image_uri=container_uri,
                                    role=role, 
                                    instance_count=2, 
                                    instance_type='ml.m5.xlarge',
                                    output_path='s3://{}/{}/model-artifacts'.format(default_bucket, prefix),
                                    sagemaker_session=sagemaker_session,
                                    base_job_name='reorder-classifier')

xgb.set_hyperparameters(objective='binary:logistic',
                        num_round=100)

In [ ]:
xgb.fit({'train': train_set_pointer, 'validation': validation_set_pointer})

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=2,
                           instance_type='ml.m5.xlarge')

### Real time inference using the deployed endpoint
<a id='real-time-inference' />

Lets get a record from test data and test the inference

In [ ]:
csv_serializer = CSVSerializer()
endpoint_name = xgb_predictor.endpoint_name
predictor = Predictor(endpoint_name=endpoint_name, 
                      serializer=csv_serializer)

In [ ]:
test_df = pd.read_csv('.././data/retrain/test.csv')
record = test_df.sample(1)
record

In [ ]:
X = record.values[0]
payload = X[1:]
payload

In [ ]:
%%time

predicted_class_prob = predictor.predict(payload).decode('utf-8')
if float(predicted_class_prob) < 0.5:
    logger.info('Prediction (y) = Will not reorder')
else:
    logger.info('Prediction (y) = Will reorder')

### Cleanup

Now that we have seen how features can be added to feature groups, it is time to delete unwated resources to not inciur charges

In [ ]:
describe_feature_group_result = sagemaker_runtime.describe_feature_group(
    FeatureGroupName=customers_feature_group_name
)
pretty_printer.pprint(describe_feature_group_result)

Delete the S3 artifacts for the offline store

In [ ]:
s3_uri = describe_feature_group_result['OfflineStoreConfig']['S3StorageConfig']['ResolvedOutputS3Uri']
full_prefix = '/'.join(s3_uri.split('/')[3:])

In [ ]:
bucket = s3.Bucket(default_bucket)
offline_objects = bucket.objects.filter(Prefix=full_prefix)
offline_objects.delete()

Delete the feature group

In [ ]:
customers_feature_group.delete()

Delete the endpoint

In [ ]:
response = sagemaker_runtime.describe_endpoint_config(EndpointConfigName=endpoint_name)
model_name = response['ProductionVariants'][0]['ModelName']
model_name

In [ ]:
sagemaker_client.delete_model(ModelName=model_name)  

In [ ]:
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

In [ ]:
sagemaker_client.delete_endpoint_config(EndpointConfigName=endpoint_name)